<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/3-LangChain/3_2_OpenAI_Moderation_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>3.2-Create a Moderation System using LangChain.</h2>
    <h3>OpenAI version</h3>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)

________
Model: gpt-4o / gpt-4o-mini / gpt-3.5-turbo

Colab Environment: CPU.

Keys:
* Lagchain
* OpenAI
* Moderator.

Article Related: [Create a Self-Moderated Commentary System with LangChain and OpenAI](https://pub.towardsai.net/create-a-self-moderated-commentary-system-with-langchain-and-openai-406a51ce0c8d?sk=b4903b827e44642f7f7c311cebaef57f)
________


# How To Create a Moderation System Using LangChain.

We are going to create a moderation system based on two models.

The first model reads user comments and generates responses.

The second language model then analyzes the generated response, identifying any negativity and modifying the response if necessary.

This process aims to prevent negative or inappropriate user input from triggering a similarly negative or off-tone response from the comment system.

In [ ]:
#Install de LangChain and openai libraries.
!pip install -q langchain==0.1.4
!pip install -q langchain-openai==0.0.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.
* OpenAI:  To interact with the OpenAI models.

In [ ]:
#PrompTemplate is a custom class that provides functionality to create prompts
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [ ]:
import torch
import os
import numpy as np

We need an OpenAI key to interac with the OpenAI API.

Here you can acces to your keys.
https://platform.openai.com/account/api-keys

OpenAI it's a pay service and you need a credit card to get a Key. But is a a relly cheap service if you only want to do some test like the ones in this notebook.

I'm using the gpt-3.5 / gpt-4o-mini as a moderator.


In [ ]:
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")


OpenAI API Key: ··········


In [ ]:
#model="gpt-3.5-turbo"
model = "gpt-4o-mini"
assistant_llm = ChatOpenAI(model=model)

Create the template for the first model called **assistant**.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment.

I included the sentiment to facilitate the creation of rude or incorrect answers.

In [ ]:
# Instruction how the LLM must respond the comments,
assistant_template = """
You are {sentiment} assistant that responds to user comments,
using similar vocabulary than the user.
User:" {customer_request}"
Comment:
"""

In [ ]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

Now we create a First Chain. Just chaining the ***assistant_prompt_template*** and the model. The model will receive the prompt generated with the ***prompt_template***.  

In [ ]:
output_parser = StrOutputParser()
assistant_chain = assistant_prompt_template | assistant_llm | output_parser


To execute the chain created it's necessary to call the **.run** method of the chain, and pass the variables necessaries.

In our case: *customer_request* and *sentiment*.

In [ ]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #calling the .invoke method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model, and to obtain it we are changing the sentiment.

In [ ]:
# This is the customer request, or customer comment in the forum moderated by the agent.
# feel free to modify it.
customer_request = """This product is a piece of shit. I feel like an Idiot!"""


In [ ]:
# Our assistant working in 'nice' mode.
response_data=create_dialog(customer_request, "nice")
print(f"assistant response: {response_data}")

assistant response: "I'm really sorry to hear that you're feeling this way! It sounds super frustrating to deal with a product that doesn't meet your expectations. You definitely don't deserve to feel like an idiot for trying it out."


The answer obtained is really polite. It dosn't need moderation.

In [ ]:
#Our assistant running in rude mode.
response_data = create_dialog(customer_request, "most rude")
print(f"assistant response: {response_data}")

assistant response: "Wow, if this product is a piece of shit, then you really did feel like an idiot for getting it. What a waste!"


As you can see the answers we obtain are not polite and we can't publish this messages to the forum, especially if they come from our company's AI assistant.

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.


In [ ]:
#The moderator prompt template
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive a Original comment and if it is impolite you must transform in polite.
Try to mantain the meaning when possible,

If it it's polite, you will let it remain as is and repeat it word for word.
Original comment: {comment_to_moderate}
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template,
)

In [ ]:
#I'm going to use a more advanced LLM
moderator_llm = ChatOpenAI(model="gpt-4o")

In [ ]:
#We build the chain for the moderator.
moderator_chain = moderator_prompt_template | moderator_llm | output_parser

In [ ]:
# To run our chain we use the .run() command
moderator_data = moderator_chain.invoke({"comment_to_moderate": response_data})

In [ ]:
print(moderator_data)

Transformed comment: "Wow, if this product didn't meet your expectations, it must have been disappointing to purchase it. What a letdown!"


Maybe the message is not perfect, but for sure that is more polite than the one produced by the ***rude assistant***.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.

First we create two chain, one for each pair of prompt and model.


It's necessary to indicate the chains and the parameters that we shoud pass in the **.invoke** method.

In [ ]:
assistant_moderated_chain = (
    {"comment_to_moderate":assistant_chain}
    |moderator_chain
)

Lets use our Moderating System!

In [ ]:
# We can now run the chain.
from langchain.callbacks.tracers import ConsoleCallbackHandler
assistant_moderated_chain.invoke({"sentiment": "impolite", "customer_request": customer_request},
                                 config={'callbacks':[ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate>] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence > 4:prompt:PromptTemplate] Entering Prompt run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/end] [1:chain:RunnableSequence > 2:chain

'Transformed comment: "It sounds like you had a frustrating experience with this purchase. Perhaps paying closer attention next time might help. Not every product will meet expectations, unfortunately."'

Every time you execute this function you can get different messages, but for sure than the one in the ***Finished Chain*** generated by our ***moderator*** is more suitable than the one in Original Comment generated by our ***rude assistant***.